# Brest Cancer Wisconsin Dataset Example

This Jupyter Notebook demonstrates how to build and use a neural network with dense layers to process and classify data.

In [ ]:
import numpy as np
from ucimlrepo import fetch_ucirepo

from src import NeuralNetwork, Config

from src.preprocessing import min_max_scaler, train_test_split
from src.activation import LeakyRelu, Sigmoid
from src.loss import BinaryCrossentropy
from src.optimizer import Adam
from src.scheduler import CosineScheduler
from src.config import FeedForwardConfig, TrainingConfig
from src.structure import Dense, Dropout

In [ ]:
# The dataset is too small to use GPU
Config.set_default_device("cpu")

breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

X = X.to_numpy()
y = y.to_numpy(dtype=str)

In [ ]:
classes = tuple(np.unique(y))

# Normalize data
data = min_max_scaler(X, 0, 1)

# Split data into training and testing sets
data_train, excepted_train, data_test, expected_test = train_test_split(data, y, random_state=0)

In [ ]:
config = FeedForwardConfig(
    network_structure=[
        Dense(len(X[0])),
        Dense(32),
        Dropout(0.2),
        Dense(16),
        Dense(8),
        Dense(1),
    ],
    classes=classes,
    hidden_activation=LeakyRelu(),
    output_activation=Sigmoid(),
)

nn = NeuralNetwork(config)

nn.train(
    data_train,
    excepted_train,
    data_test,
    expected_test,
    config=TrainingConfig(
        lr=CosineScheduler(
            learning_rate=0.01, min_lr=0.0001, max_steps=100, cyclic=True
        ),
        patience_stop=100,
        loss=BinaryCrossentropy(),
        optimizer=Adam(),
        batch_size=4,
        debug=True,
        store=None,
    ),
)

print(
    f"final test: {nn.evaluate(data, y)},"
    f"test: {nn.evaluate(data_test, expected_test)},"
    f"train: {nn.evaluate(data_train, excepted_train)}"
)